## 決策樹

In [ ]:
#準備引入存在google drive上的資料夾
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#引入蝦皮資料集
import re
import pandas as pd
try:
    df = pd.read_csv("/content/drive/MyDrive/shopee_item_category.csv")
    # Continue processing the content
except UnicodeDecodeError as e:
    # Handle the UnicodeDecodeError by specifying a different encoding or replacing invalid characters
    print(f"UnicodeDecodeError: {e}")
except FileNotFoundError:
    print("File not found. Check the file path.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

In [ ]:
#引入前處理過的蝦皮資料集商品名稱
df1 = pd.read_csv("/content/drive/MyDrive/output.csv")
#將兩份資料集合併，方便之後提取前處理過的商品名稱和原有的大類、小類欄位
merged_df = pd.concat([df, df1], axis=1)
data=merged_df

In [ ]:
#引入需要的套件
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [ ]:
#各訓練10次大類、小類的決策樹分類模型，一次隨機取15000筆資料
for i in range(10):
  classified_data = data.sample(n=15000)
  classified_data['name'] = data['name'].fillna('')
  tfidf_vectorizer = TfidfVectorizer()
  X = tfidf_vectorizer.fit_transform(classified_data['name'])

# 標籤編碼
  label_encoder_big = LabelEncoder()
  label_encoder_small = LabelEncoder()

  y_big = label_encoder_big.fit_transform(classified_data['大類'])
  y_small = label_encoder_small.fit_transform(classified_data['小類'])

# 分割資料集
  X_train, X_test, y_train_big, y_test_big, y_train_small, y_test_small = train_test_split(
      X, y_big, y_small, test_size=0.2, random_state=42
  )
#大類訓練測試
  clf_big = DecisionTreeClassifier()
  clf_big.fit(X_train, y_train_big)
  pred=clf_big.predict(X_test)
  zipped_arrays = zip(y_test_big, pred)
#小類訓練測試
  clf_small = DecisionTreeClassifier()
  clf_small.fit(X_train, y_train_small)
  pred1=clf_small.predict(X_test)
  zipped_arrays1 = zip(y_test_small, pred1)
#計算測試出來的正確率
  common_elements_count = sum(x == y for x, y in zipped_arrays)
  common_elements_count1 = sum(x == y for x, y in zipped_arrays1)
  print("大類準確率",common_elements_count/pred.size,"小類準確率",common_elements_count1/pred1.size)

大類準確率 0.5913333333333334 小類準確率 0.30466666666666664
大類準確率 0.6006666666666667 小類準確率 0.288
大類準確率 0.6163333333333333 小類準確率 0.31
大類準確率 0.601 小類準確率 0.306
大類準確率 0.6023333333333334 小類準確率 0.31533333333333335
大類準確率 0.6053333333333333 小類準確率 0.323
大類準確率 0.6073333333333333 小類準確率 0.31466666666666665
大類準確率 0.599 小類準確率 0.2966666666666667
大類準確率 0.5993333333333334 小類準確率 0.31266666666666665
大類準確率 0.6283333333333333 小類準確率 0.33166666666666667


10次共執行15分鐘
大類準確率平均約6成
小類準確率平均約3成

In [ ]:
# 讀入未分類的資料集（name經前處理
unclassified_data = pd.read_csv("/content/drive/MyDrive/output3.csv")  # 替換為實際的檔案路徑data_a = pd.read_csv("/content/drive/MyDrive/output3.csv")
#去除有缺失值的資料
cleaned_data = unclassified_data.dropna(subset=['name']).copy()

In [ ]:
#預測大類
X_unclassified = tfidf_vectorizer.transform(cleaned_data['name'])
predictions_big = clf_big.predict(X_unclassified)
#預測小類
X_unclassified_small = tfidf_vectorizer.transform(cleaned_data['name'])
predictions_small = clf_small.predict(X_unclassified)

In [ ]:
# 將預測結果轉換回原始的類別名稱
predicted_big_category = label_encoder_big.inverse_transform(predictions_big)
predicted_small_category = label_encoder_small.inverse_transform(predictions_small)

# 將預測結果添加到未分類的商品資料集
#cleaned_data['預測大類'] = predicted_big_category
#cleaned_data['預測小類'] = predicted_small_category
# 在原始資料框上使用 .loc 賦值，避免SettingWithCopyWarning
cleaned_data.loc[:, '預測大類'] = predicted_big_category
cleaned_data.loc[:, '預測小類'] = predicted_small_category

# 保存結果
cleaned_data.to_csv('cleaned_data_with_predictions_decision_tree.csv', index=False)


In [ ]:
c=pd.read_csv('cleaned_data_with_predictions_decision_tree.csv')

In [ ]:
c[457:460]

,Unnamed: 0,name,預測大類,預測小類
457,457,日本浴室除霉,書籍及雜誌期刊,其他
458,458,NIKE DUNK 白銀斷勾DJ,男鞋,休閒鞋款
459,459,博士清潔錠,書籍及雜誌期刊,其他


In [ ]:
c[674:675]

,Unnamed: 0,name,預測大類,預測小類
674,675,純水柔濕巾,書籍及雜誌期刊,濕紙巾


以上部分資料為例：

大類多被分到書籍、小類多被分到其他

但也有少數分對大or小類甚至全對
